In [ ]:
import pandas as pd
import numpy as np
from tensorflow import keras
from keras import layers, models, callbacks
import string
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sentence_transformers import SentenceTransformer
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
import os

# Phishing E-Mail Detector
Dieses Notebook enthält ein Modell, das vorhersagen kann, ob ein gegebenes Embedding einer E-Mail eine Phishing-Mail ist oder sicher ist. Um dies zu erreichen, lädt es zuvor erstellte Embeddings von E-Mails, die entweder als Phishing oder sicher klassifiziert wurden. Anschließend werden die Daten in Trainings-, Validierungs- und Testdatensätze aufgeteilt, bevor ein Modell mit den Daten definiert und trainiert wird. Schließlich wird die Leistung des Modells bewertet. Die grundlegende Struktur orientiert sich an CRISP-DM.
## CRISP-DM Phasen

1. Business Understanding
2. Data Understanding
3. Data Preparation
4. Modeling
5. Evaluation
6. Deployment


## Kontext
### Georgiev International GmbH 
Kleines Eisenguss Unternehmen, welches sich vor einem Jahr seine IT-Infrastruktur aufgesetzt hat. Leider erhält dieses Unternehmen viele Phishing Mails aus Bulgarien. Daher wurden die Studenten der DHBW beauftragt einen Phishing Filter für deren Gmail Accounts zu entwickeln.

### 1. Business Understanding

##### Aufgbabe
Entwicklung eines Modells, das vorhersagen kann, ob eine Email Phishing ist oder nicht.

##### Ziele
- Reduzieren der Anzahl an Phishing Mails, die in der Inbox des Nutzers landen.
- Minimieren von False Positives um sicherzustellen, dass legitime Emails nicht fälschlicherweise als Phishing gekennzeichnet werden.
- Verbessern des allgemeinen Nutzervertrauens

##### Erfolgskriterien
- Erreichen einer Genauigkeit von über 90%
- False Positive Rate unter 5%
- False Negative Rate unter 2%

##### Stakeholder
- Geschäftsführer der Georgiev International GmbH "Marco"

##### Benötigt werden
- Zugang zu historischen Email-Daten mit gelabelten Beispielen von Phishing und Non-Phishing Emails.
- Infrastruktur zum Entwickeln, Testen und Produktiv Setzen des Modells.

### 2. Data Understanding
Die Daten kommen aus folendem Datensatz: https://www.kaggle.com/datasets/subhajournal/phishingemails/data. 

- 18.600 E-Mails im Datensatz
- 61% sicher, 39% Phishing
- Moderates Ungleichgewicht
- Gefahr von Modell-Bias zugunsten sicherer E-Mails
- 3% leere E-Mails

Der Datensatz enthält drei Spalten:
    
1. Die Anzahl an Einträgen
2. Den Email Text
3. Das Email Label

In [ ]:
data = pd.read_csv("data/Phishing_Email.csv", skiprows=0)

print(f"The amount of data entries is: {len(data)}\n")
print(data.head())

### 3. Data Preparation

#### Entfernen von leeren Datensätzen

In [ ]:
# Remove rows with missing text
data = data.dropna(subset=["Email Text"])

#### Lösen des Data Imbalance Problems durch Löschen überschüssiger Safe Emails
Wie bereits im Data Understanding beschrieben liegt nur ein moderates Ungleichgewicht vor, ein Lösen des Data Imbalance Problems durch das Löschen überschüssiger Safe Emails macht also ggf. keinen signifikanten Unterschied. 

Zum Vergleich: 

F-Wert ohne Löschen = 0.9167

F-Wert mit Löschen = 0.9465

Ein Löschen der überschüssigen Daten zeigt also Vorteile, weswegen es hier gemacht wird.

In [ ]:
excess_safe_mails = data[data['Email Type'] == 'Safe Email'].shape[0] - data[data['Email Type'] == 'Phishing Email'].shape[0]
print(f"Excess Safe Mails before resolving data imbalance: {excess_safe_mails}")

# Filter the DataFrame to get rows with the specified value
filtered_df = data[data['Email Type'] == 'Safe Email']

# Randomly select entries to remove
rows_to_remove = filtered_df.sample(n=excess_safe_mails, random_state=1).index

# Drop the selected entries from the original DataFrame
data = data.drop(rows_to_remove)

excess_safe_mails = data[data['Email Type'] == 'Safe Email'].shape[0] - data[data['Email Type'] == 'Phishing Email'].shape[0]
print(f"Excess Safe Mails after resolving data imbalance: {excess_safe_mails}")
n_safe_mails = data[data['Email Type'] == 'Safe Email'].shape[0]
n_phishing_mail = data[data['Email Type'] == 'Phishing Email'].shape[0]
print(f"Number of Safe Mails: {n_safe_mails}, Number of Phishing Mails: {n_phishing_mail}")

#### Datenaufbereitung
Hier werden folgende Datenaufbereitungsschritte unternommen
- Alle Buchstaben zu Kleinbuchstaben machen
- Entfernen von Punktzeichen 
- Entfernen von Stopwords
- Tokenization
- Stemming
- Lemmatization

In [ ]:
def clean_texts(texts):
    # Convert to lowercase
    texts = [text.lower() for text in texts]

    # Remove punctuation
    texts = [text.translate(str.maketrans("", "", string.punctuation)) for text in texts]

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    texts = [' '.join(word for word in text.split() if word not in stop_words) for text in texts]

    # Tokenize
    texts = [word_tokenize(text) for text in texts]

    # Stemming
    stemmer = PorterStemmer()
    texts = [[stemmer.stem(word) for word in text] for text in texts]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    texts = [[lemmatizer.lemmatize(word) for word in text] for text in texts]

    # Join tokens back into strings
    texts = [' '.join(text) for text in texts]

    return texts

#### Erstellen der Embeddings
- Nutzen des Sentence Transformer Modells allMiniLM (https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2)
- Beim Ersten Durchlaufen des Notebooks werden diese in einer Datei gespeichert, die dann in weiteren Durchläufen ausgelesen wird um die Performance zu verbessern.

In [ ]:
def get_embeddings(texts):
    # Load the sentence transformer model
    sentence_transformer_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
    
    # Preprocess the data
    cleaned_texts = clean_texts(texts)

    # Encode the text data to get embeddings
    embeddings = sentence_transformer_model.encode(cleaned_texts, show_progress_bar=True)

    # Convert embeddings to a list of lists with Python floats
    embeddings_list = [list(map(float, embedding)) for embedding in embeddings]

    # Convert embeddings to a single string representation
    embeddings_str_list = [str(embedding) for embedding in embeddings_list]
    
    return embeddings_str_list

# Check if the embeddings exist and if not, create them
if not os.path.exists("data/embeddings.csv"):
    # Create a new DataFrame with a single column for embeddings
    embeddings_df = pd.DataFrame({"embedding": get_embeddings(data["Email Text"].tolist())})

    # Concatenate the label column with the embeddings
    final_df = pd.concat([data["Email Type"].reset_index(drop=True), embeddings_df], axis=1)
    
    # Save the final DataFrame to a CSV file
    final_df.to_csv("data/embeddings.csv", index=False, sep=";")

# Load the embeddings and skip the header
data_embeddings = pd.read_csv('data/embeddings.csv', sep=';', skiprows=0)

#### Erstellen eines Trainings, Validations und Testdatensatzes
Dazu wird die Aufteilung 60% Trainingsdate, 20% Validationsdaten und 20% Testdaten genutzt

In [ ]:
# Specifying the test split is not necessary for the code, but is done anyway for clarity
def format_and_load_data(train_split=0.6, validate_split=0.2, test_split=0.2):
    # Calculate the number of samples for each dataset split
    train_amount = int(len(data) * train_split)
    validate_amount = int(len(data) * validate_split)
    
    examples = []

    # Process each embedding
    for _, embedding in data_embeddings.iterrows():
        label = 1 if embedding.iloc[0] == 'Phishing Email' else 0
        embedding_list = [float(i) for i in embedding.iloc[1].strip('[]').split(', ')]
        examples.append((embedding_list, label))

    # Shuffle the examples
    np.random.shuffle(examples)

    # Split data into features and labels
    X, y = zip(*examples)

    # Convert lists to numpy arrays
    X = np.array(X)
    y = np.array(y)

    # Split data into training, validation and testing sets
    X_train = X[:train_amount]
    X_val = X[train_amount:train_amount + validate_amount]
    X_test = X[train_amount + validate_amount:]
    y_train = y[:train_amount]
    y_val = y[train_amount:train_amount + validate_amount]
    y_test = y[train_amount + validate_amount:]

    return X_train, y_train, X_val, y_val, X_test, y_test

### 4. Modeling

#### Erstellen eines binären Klassifizierungsmodells, um vorherzusagen, ob eine E-Mail Phishing ist.

Das Modell besteht aus drei Schichten:
- Die erste Schicht akzeptiert Eingaben mit der Länge der erstellten Einbettungen (hier 384).
- Die zweite Schicht akzeptiert Eingaben mit der Länge 128.
- Die dritte Schicht akzeptiert Eingaben mit der Länge 1.

- Ersten beiden Schichten verwenden die ReLU-Aktivierungsfunktion
- Letzte Schicht verwendet Sigmoid.
    - Sigmoid wurde gewählt, weil es Werte zwischen 0 und 1 ausgibt, was es für binäre Klassifizierungsaufgaben geeignet macht. (Es wird auch sehr häufig für diese Art von Aufgaben verwendet.)

Während des Trainings werden die Validierungsdaten und der Validierungsverlust verwendet, um das Modell nach jeder Epoche zu bewerten. Die Validierungsgenauigkeit wurde ebenfalls berücksichtigt, zeigte jedoch schwächere Ergebnisse als die Verlustmetrik.

Um den Trainingsprozess effektiv zu verwalten, werden zwei Callback-Funktionen verwendet:
- **Early Stopping**: Stoppt das Training, wenn die Leistung des Modells, gemessen am Validierungsdatensatz, zu verschlechtern beginnt, um Überanpassung zu verhindern.
- **Reduce Learning Rate on Plateau**: Passt die Lernrate an (in diesem Fall, indem sie halbiert wird), wenn die Leistung des Modells zu sinken beginnt, um zu verhindern, dass es in lokalen Minima stecken bleibt. 

    Der Patience-Parameter für beide Callbacks definiert die Anzahl der Epochen, die sie abwarten, bevor sie Maßnahmen ergreifen.

In [ ]:
# Load the data splits
X_train, y_train, X_val, y_val, X_test, y_test = format_and_load_data()

# Reshape data to make it acceptable for the model
y_train = np.array(y_train, dtype=np.int32)
y_val = np.array(y_val, dtype=np.int32)
y_test = np.array(y_test, dtype=np.int32)

print(X_train.shape, X_train[0].shape)

# Define model architecture
no_embedding_dim = len(X_train[0])
model = models.Sequential([
    layers.Dense(no_embedding_dim, activation='relu', input_shape=(no_embedding_dim,)),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Define callbacks
early_stopping = callbacks.EarlyStopping(patience=5, monitor='val_loss', restore_best_weights=True)
reduce_lr = callbacks.ReduceLROnPlateau(factor=0.5, patience=2, monitor='val_loss', verbose=1)

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping, reduce_lr])

# Save the model for later use
model.save("models/phishing_email_classifier.h5")

model_summary = model.summary()
model_summary

### 5. Evaluation

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

#### Loss and accuracy
- **Verlust (Loss)**: Misst die Differenz zwischen vorhergesagten und tatsächlichen Werten.

- **Genauigkeit (Accuracy)**: Gibt an, wie oft das Modell korrekte Vorhersagen trifft.

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')

#### F1-Score
Der F1-Wert berechnet sich als harmonisches Mittel zwischen Precision und Recall, die jeweils wie folgt berechnet werden:

- **Precision**: Die Precision misst das Verhältnis der korrekt als positiv vorhergesagten Instanzen zur Gesamtanzahl der positiv vorhergesagten Instanzen. Sie wird berechnet als:
  $$
  \text{Precision} = \frac{TP}{TP + FP}
  $$
  Dabei sind \(TP\) die richtig positiven Vorhersagen (true positives) und \(FP\) die falsch positiven Vorhersagen (false positives).

- **Recall**: Der Recall (auch Sensitivität genannt) misst das Verhältnis der korrekt als positiv vorhergesagten Instanzen zur Gesamtanzahl der tatsächlich positiven Instanzen. Er wird berechnet als:
  $$
  \text{Recall} = \frac{TP}{TP + FN}
  $$

  Hierbei sind \(TP\) die richtig positiven Vorhersagen und \(FN\) die falsch negativen Vorhersagen (false negatives).

- **F1-Score**:
  $$
  \text{F1-Score} = 2 \cdot \frac{\text{Precision} \cdot \text{Recall}}{\text{Precision} + \text{Recall}}
  $$

In [ ]:
y_pred_prob = model.predict(X_test).flatten()
y_pred = (y_pred_prob > 0.5).astype(np.int32)

precision = precision_score(y_test, y_pred)
print(f'Precision: {precision:.4f}')

recall = recall_score(y_test, y_pred)
print(f'Recall: {recall:.4f}')

f1 = f1_score(y_test, y_pred)
print(f'F1 Score: {f1:.4f}')

#### ROC AUC Score

- **ROC-AUC (Receiver Operating Characteristic - Area Under the Curve)**: Der ROC-AUC-Wert misst die Fähigkeit eines binären Klassifikators, zwischen positiven und negativen Klassen zu unterscheiden, unabhängig von der gewählten Klassifikationsschwelle.

  Der ROC-AUC-Wert wird berechnet, indem die Fläche unter der ROC-Kurve betrachtet wird. Die ROC-Kurve zeigt das Verhältnis von True Positive Rate (TPR) zu False Positive Rate (FPR) für verschiedene Klassifikationsschwellen. Ein höherer ROC-AUC-Wert (normalerweise im Bereich von 0 bis 1) deutet auf ein besseres Modell hin, das besser zwischen den Klassen unterscheiden kann.

  Eine ROC-AUC von 0,5 bedeutet, dass das Modell nicht besser als zufällige Vorhersagen ist, während ein Wert nahe 1 zeigt, dass das Modell eine sehr gute Trennung zwischen den Klassen erreicht hat.


In [ ]:
roc_auc = roc_auc_score(y_test, y_pred_prob)
print(f'ROC AUC Score: {roc_auc:.4f}')

#### Confusion Matrix

- **True Positives (TP)**: Korrekt als positiv vorhergesagte Instanzen.
- **False Positives (FP)**: Fälschlicherweise als positiv vorhergesagte Instanzen.
- **True Negatives (TN)**: Korrekt als negativ vorhergesagte Instanzen.
- **False Negatives (FN)**: Fälschlicherweise als negativ vorhergesagte Instanzen.


Ein Beispiel für die Darstellung einer Confusion Matrix:

|                 | Predicted Negative | Predicted Positive |
|-----------------|--------------------|--------------------|
| **Actual Negative** | TN                 | FP                 |
| **Actual Positive** | FN                 | TP                 |


In [ ]:
conf_mat = confusion_matrix(y_test, y_pred)

# Extracting elements from the confusion matrix
TN, FP, FN, TP = conf_mat.ravel()

# Calculating false positive rate and false negative rate
Total_Predictions = FP + FN + TP + TN
False_Positive_Rate = FP / Total_Predictions
False_Negative_Rate = FN / Total_Predictions

print(f"False Positive Rate: {False_Positive_Rate}")
print(f"False Negative Rate: {False_Negative_Rate}")

plt.figure(figsize=(8, 6))
sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues', cbar=False,
            annot_kws={'fontsize': 14, 'fontweight': 'bold'})
plt.xlabel('Predicted Labels', fontsize=14)
plt.ylabel('True Labels', fontsize=14)
plt.title('Confusion Matrix', fontsize=16)
plt.xticks(ticks=[0.5, 1.5], labels=['Not Phishing', 'Phishing'])
plt.yticks(ticks=[0.5, 1.5], labels=['Not Phishing', 'Phishing'])
plt.show()

### 6. Deployment
Das Modell kann zur Nutzung aus dem models Ordner geladen und dann genutzt werden. Nachfolgend ist beispielhaft die Nutzung des Modells gezeigt.

In [ ]:
# Load the saved model for phishing email classification
model = keras.models.load_model("models/phishing_email_classifier.h5")

# Creating sample mails on my own to test and demonstrate the model
mails = ["You won the lottery! Please send us your bank account details to claim your prize.",
         "Hello I am a Nigerian prince and I want to share my fortune with you. Please send me your bank account details so I can transfer the money to you.",
         "Hello I am just confirming that I got your previous email and I will get back to you soon. Thanks"]

# Encode the preprocessed email text to get its embedding
email_embeddings = get_embeddings(mails)

email_embeddings_formatted = []
# Process each embedding
for ee in email_embeddings:
    embedding_list = [float(i) for i in ee.strip('[]').split(', ')]
    email_embeddings_formatted.append(embedding_list)
    
email_embeddings_formatted = np.array(email_embeddings_formatted)

# Make predictions for each embedding
predictions = model.predict(email_embeddings_formatted)

# Thresholding the predictions
threshold = 0.5
predicted_labels = ["Phishing" if prediction[0] >= threshold else "Safe" for prediction in predictions]

# Print predictions for each mail
for mail, prediction in zip(mails, predicted_labels):
    print(f"Mail: {mail}\nPrediction: {prediction}\n")